In [5]:
import os
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [6]:
import ast
import warnings
import re
warnings.filterwarnings("ignore")

In [7]:
IMG_DIR = "../coco_dataset_val_2017_all/imgs/"
CAPS_DIR = "../coco_dataset_val_2017_all/caps/"

CLASSES = {
 0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mouse',
 65: 'remote',
 66: 'keyboard',
 67: 'cell phone',
 68: 'microwave',
 69: 'oven',
 70: 'toaster',
 71: 'sink',
 72: 'refrigerator',
 73: 'book',
 74: 'clock',
 75: 'vase',
 76: 'scissors',
 77: 'teddy bear',
 78: 'hair drier',
 79: 'toothbrush',
 80: 'toothbrush'
 }

def get_class_name(class_id):
    return CLASSES[class_id]

def get_caption(img_id, caps_dir):
    with open(f"{caps_dir}{img_id}.txt", "r") as f:
        return f.read()

In [8]:
model = YOLO("yolov8n.pt")

In [9]:
results = model.predict(source=IMG_DIR, save_txt=True, save=True, conf=0.5)



    WARNING  stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

image 1/5000 C:\Users\JARVIS\OneDrive\Desktop\Aaryan_Image_captioning\coco_dataset_val_2017_all\imgs\1000.jpg: 480x640 10 persons, 1550.5ms
image 2/5000 C:\Users\JARVIS\OneDrive\Desktop\Aaryan_Image_captioning\coco_dataset_val_2017_all\imgs\100238.jpg: 480x640 4 persons, 3 frisbees, 614.9ms
image 3/5000 C:\Users\JARVIS\OneDrive\Desktop\Aaryan_Image_captioning\coco_dataset_val_2017_all\imgs\100274.jpg: 448x640 1 car, 1 train, 639.8ms
image 4/5000 C:\Users\JA

In [10]:
for result, cap in zip(results, os.listdir(CAPS_DIR)):
    box = result.boxes
    img_id = cap.split(".")[0]

    xyxy = None
    class_name = None
    confidence = None

    with open(CAPS_DIR + cap, "r") as f:
        caption = f.read()

    all_captions = caption.split("\n")[:-1]
    
    res = []
    for ele in all_captions:
        if ele.strip():
            res.append(ele)

    all_captions = res.copy()

    if len(all_captions) != 5:
        all_captions = all_captions[:5]
        print(f"Image ID: {img_id} NOW has {len(all_captions)} captions")
        print(all_captions)

    
    for i in range(len(box)):
        xyxy = box.xyxy.tolist()[i]
        xyxy = [round(x, 2) for x in xyxy]
        class_name = get_class_name(box.cls.tolist()[i])
        confidence = box.conf.tolist()[i]

        with open("../data/x_new.txt", "a") as f:
            f.write(f"{class_name}: {xyxy}, ")   

    with open("../data/x_new.txt", "a") as f:
        f.write(f"\n")

    with open("../data/y_new.txt", "a") as f:
        f.write(f"{all_captions} \n")

Image ID: 163057 NOW has 5 captions
['A group of people flying kites on a grassy field next to buildings.', 'Many people walk through a park as few kites fly in the air.', 'People are flying kites in the park and walking.', 'People fly kites in a large park in the middle of a city.', 'People flying kites in a park on a windy day.']
Image ID: 165257 NOW has 5 captions
['A large black counter top sitting next to a sink.', 'a clean kitchen counter with a clean sink', 'A kitchen with a sink, dishwasher and some boxes on the counter.', 'A kitchen with a sink, dishwasher and boxes on the counter.', 'a black counter on a wood cabinet in a kitchen']
Image ID: 190841 NOW has 5 captions
['A man on a skateboard jumps over a ramp.', 'a person jumping a skate board into the air', 'A skateboarder jumping off a small ramp places on the street.', 'A man in black shirt doing a trick on a skateboard.', 'A man attempts a trick after riding off of a low ramp.']
Image ID: 215259 NOW has 5 captions
['Two wo

KeyboardInterrupt: 

In [11]:
def draw_bbox(img, coord_list, color=(0, 255, 0), thickness=2):
    for i in range(len(coord_list)):
        x1, y1, x2, y2 = coord_list[i]
        cv2.rectangle(img, (x1, y1), (x2, y2),
                      color=color, thickness=thickness)


def show_img(img, figsize=(10, 10)):
    plt.figure(figsize=figsize)
    plt.imshow(img) 
    plt.show()


def show_img_bbox(img, coord_list, figsize=(5, 5)):
    draw_bbox(img, coord_list)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    show_img(img, figsize=figsize)

# Crop the object and find its median color


def crop_object(img, coord_list):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    crop_img = []
    for i in range(len(coord_list)):
        x1, y1, x2, y2 = coord_list[i]
        crop_img.append(img[y1:y2, x1:x2])
    return crop_img


def find_median_color(img):
    median_color = []
    for i in range(len(img)):
        median_color.append(np.median(img[i], axis=(0, 1)))
    return median_color


def get_color_name(rgb):
    colors = {
        "red": (255, 0, 0),
        "green": (0, 255, 0),
        "blue": (0, 0, 255),
        "yellow": (255, 255, 0),
        "magenta": (255, 0, 255),
        "cyan": (0, 255, 255),
        "black": (0, 0, 0),
        "white": (255, 255, 255),
        "orange": (255, 165, 0),
        "purple": (128, 0, 128),
        "pink": (255, 192, 203),
        "brown": (165, 42, 42),
        "gold": (255, 215, 0),
        "silver": (192, 192, 192),
        "gray": (128, 128, 128),
        "lavender": (230, 230, 250),
        "olive": (128, 128, 0),
        "teal": (0, 128, 128),
        "navy": (0, 0, 128),
        "maroon": (128, 0, 0)
    }

    min_distance = float("inf")
    closest_color = None
    for color, value in colors.items():
        distance = sum([(i - j) ** 2 for i, j in zip(rgb, value)])
        if distance < min_distance:
            min_distance = distance
            closest_color = color
    return closest_color


def color_from_median(img, coord_list,nested_list):
    crop_img = crop_object(img, coord_list)
    # print('LEN_CROP=',len(crop_img))
    median_color = find_median_color(crop_img)
    median_color = [tuple([int(x) for x in color]) for color in median_color]
    # print('MEDIAN_COLOR===',median_color)
    closest_colors = []
    merge_list=list(zip(nested_list,median_color))
    # print(merge_list)
    for obj,color in merge_list:
        color_img = np.zeros((1, 1, 3), np.uint8)
        color_img[:] = color
        # show_img(color_img, figsize=(1, 1))
        closest_color = get_color_name(color)
        # print(f"{obj}: {closest_color}")
        closest_colors.append(obj+':'+closest_color)

    return closest_colors


In [12]:

import re

nested_list = []

with open('../data/x_new.txt', 'r') as f:
    for line in f:
        line_list = re.findall(r"(\w+):\s*\[[\d.,\s]*\]", line)
        nested_list.append(line_list)

# print(nested_list)
obj_color=[]
for result in range(len(results)):
    box = results[result].boxes
    # print('BOX====',box)
    coord_list = []
    
    nested_list[result]
    # print("nested_list=",nested_list)
    for i in range(len(box)):
        xyxy = box.xyxy.tolist()[i]
        xyxy = [int(x) for x in xyxy]
        coord_list.append(xyxy)
    # print(coord_list)
    
    # show_img_bbox(result.orig_img, coord_list)
    x=closest_colors = color_from_median(results[result].orig_img, coord_list,nested_list[result])
    obj_color.append(x)
print(obj_color)    

[['person:brown', 'person:gray', 'person:gray', 'person:gray', 'person:gray', 'person:brown', 'person:gray', 'person:brown', 'person:gray', 'person:brown'], ['frisbee:silver', 'person:gray', 'person:gray', 'person:gray', 'person:gray', 'frisbee:silver', 'frisbee:gray'], ['car:gray', 'train:olive'], ['sign:black', 'car:olive'], ['person:gray', 'person:gray'], ['bird:silver'], ['horse:brown', 'person:brown', 'person:gray'], ['pizza:olive'], ['person:brown', 'person:black', 'person:olive', 'person:black', 'phone:brown'], ['person:brown', 'person:black', 'person:black', 'person:black', 'person:maroon', 'person:black', 'person:gray', 'person:black'], ['bed:gray', 'chair:brown', 'table:maroon'], [], ['person:gray', 'bat:gray'], ['couch:brown', 'cat:brown'], ['cat:gray', 'bicycle:gray'], ['giraffe:black', 'giraffe:gray', 'giraffe:brown', 'giraffe:gray'], ['airplane:silver'], ['car:gray', 'clock:gray'], ['motorcycle:gray', 'person:gray'], ['motorcycle:gray', 'person:black'], ['motorcycle:black

In [13]:
len(obj_color)

5000

In [14]:
original_list = obj_color

new_list = [' '.join([x.split(":")[1] for x in sublist]) for sublist in original_list]

print(new_list) 


['brown gray gray gray gray brown gray brown gray brown', 'silver gray gray gray gray silver gray', 'gray olive', 'black olive', 'gray gray', 'silver', 'brown brown gray', 'olive', 'brown black olive black brown', 'brown black black black maroon black gray black', 'gray brown maroon', '', 'gray gray', 'brown brown', 'gray gray', 'black gray brown gray', 'silver', 'gray gray', 'gray gray', 'gray black', 'black gray gray teal black brown black black gray', 'gray', 'maroon', 'black brown gray', 'olive brown silver', 'gray', 'brown gray brown orange', 'brown black brown black', 'gray black', 'gray', 'brown', 'gray gray', 'brown', 'silver gray', 'olive', '', 'silver', 'black', 'olive olive brown olive brown', '', 'brown', 'gray black', 'gray silver', 'maroon brown gray gray black gray gray', '', 'brown brown', '', 'brown black maroon maroon', 'black', 'gray', 'brown gray brown', 'brown', 'gray', 'pink', 'brown', 'silver', 'gray brown', 'gray', 'gray gray', 'brown maroon black maroon brown',

In [15]:
def load_cleaned_text_data():
    with open("../data/x_bert.txt", 'r') as f:
        x_text = f.readlines()

    x_text_cleaned = []
    for x in x_text:
        x_text_cleaned.append(x.split('\n')[0])

    with open("../data/y.txt", 'r') as f:
        y_file = f.readlines()

    y_list_temp = []
    for y_val in y_file:
        y_list_temp.append((y_val.split(" \n")[0]))

    y_list = []
    for y in y_list_temp:
        x = y
        x = ast.literal_eval(x)
        y_list.append(x)

    y_list_clean = []
    for y in y_list:
        for i in range(len(y)):
            y[i] = re.sub(r'[^\w\s]', '', y[i])
            y[i] = y[i].lower()
        y_list_clean.append(y)

    return x_text_cleaned, y_list_clean

x_text, y_text = load_cleaned_text_data()

In [16]:
len(x_text),len(y_text)

(5000, 5000)

In [17]:
x_text[2:5]

['car 20 50 40 60 train 80 30 80 40',
 'stop sign 30 10 70 60 car 30 80 40 90',
 'person 40 20 100 100 person 20 20 60 100']

In [18]:
input_list = x_text

output_list = []

for input_str in input_list:
    words_coords = input_str.split()
    words = []
    coords = []
    item=0
    while item < len(words_coords):
        if words_coords[item].isdigit():
            coords.append(int(words_coords[item]))
            item=item+1
        else:
            words.append(words_coords[item])
            if(words_coords[item+1].isalpha()):
                try:
                    words[-1]=words[-1]+ '_' + words_coords[item+1]
                except:
                    print(input_list.index(input_str))


                item=item+2

            else:
                item=item+1
               
    result = ''
    for i in range(len(words)):
        result += words[i] + ' '
        result += ' '.join(map(str, coords[i*4:(i+1)*4]))
        result += ' '
    output_list.append(result)
output_list[2:5]    

['car 20 50 40 60 train 80 30 80 40 ',
 'stop_sign 30 10 70 60 car 30 80 40 90 ',
 'person 40 20 100 100 person 20 20 60 100 ']

In [19]:
output_list[2:5]

['car 20 50 40 60 train 80 30 80 40 ',
 'stop_sign 30 10 70 60 car 30 80 40 90 ',
 'person 40 20 100 100 person 20 20 60 100 ']

In [20]:
my_list = output_list

for i in range(len(my_list)):
    my_list[i] = my_list[i].strip()

In [21]:
my_list[2:5]

['car 20 50 40 60 train 80 30 80 40',
 'stop_sign 30 10 70 60 car 30 80 40 90',
 'person 40 20 100 100 person 20 20 60 100']

In [73]:
l=my_list
m=new_list

In [74]:
print(l)
print(m)

['person 60 40 80 100 person 20 30 30 80 person 10 40 20 80 person 80 40 100 100 person 50 30 60 100 person 40 20 60 90 person 30 40 40 90 person 70 20 80 50 person 30 20 40 80 person 60 30 70 100', 'frisbee 40 50 60 60 person 0 0 30 100 person 30 20 70 100 person 60 0 80 100 person 80 40 100 100 frisbee 60 20 80 40 frisbee 10 50 20 60', 'car 20 50 40 60 train 80 30 80 40', 'stop_sign 30 10 70 60 car 30 80 40 90', 'person 40 20 100 100 person 20 20 60 100', 'bird 20 10 60 60', 'horse 0 20 60 90 person 30 10 40 50 person 90 40 100 60', 'pizza 0 0 100 100', 'person 10 10 50 100 person 80 20 90 60 person 50 20 60 60 person 40 20 50 60 cell_phone 30 30 40 50', 'person 40 60 70 100 person 0 60 30 100 person 90 30 100 100 person 20 50 40 100 person 50 30 70 80 person 80 60 90 100 person 40 40 50 80 person 60 40 80 100', 'bed 10 10 70 100 chair 80 70 90 100 dining_table 80 60 100 90', '', 'person 60 20 100 100 baseball_bat 60 30 70 60', 'couch 0 40 100 100 cat 10 20 50 40', 'cat 60 30 100 70 

In [75]:
new_list_l = []

for string in l:
    sub_list = []
    split_string = string.split()
    for i in range(0, len(split_string), 5):
        sub_list.append(' '.join(split_string[i:i+5]))
    new_list_l.append(sub_list)
    
print(new_list_l)

new_color_l = [[item] for item in m]

new_color_l = [s[0].split() for s in new_color_l]
print(new_color_l)


[['person 60 40 80 100', 'person 20 30 30 80', 'person 10 40 20 80', 'person 80 40 100 100', 'person 50 30 60 100', 'person 40 20 60 90', 'person 30 40 40 90', 'person 70 20 80 50', 'person 30 20 40 80', 'person 60 30 70 100'], ['frisbee 40 50 60 60', 'person 0 0 30 100', 'person 30 20 70 100', 'person 60 0 80 100', 'person 80 40 100 100', 'frisbee 60 20 80 40', 'frisbee 10 50 20 60'], ['car 20 50 40 60', 'train 80 30 80 40'], ['stop_sign 30 10 70 60', 'car 30 80 40 90'], ['person 40 20 100 100', 'person 20 20 60 100'], ['bird 20 10 60 60'], ['horse 0 20 60 90', 'person 30 10 40 50', 'person 90 40 100 60'], ['pizza 0 0 100 100'], ['person 10 10 50 100', 'person 80 20 90 60', 'person 50 20 60 60', 'person 40 20 50 60', 'cell_phone 30 30 40 50'], ['person 40 60 70 100', 'person 0 60 30 100', 'person 90 30 100 100', 'person 20 50 40 100', 'person 50 30 70 80', 'person 80 60 90 100', 'person 40 40 50 80', 'person 60 40 80 100'], ['bed 10 10 70 100', 'chair 80 70 90 100', 'dining_table 80 6

In [76]:
print(new_color_l)

[['brown', 'gray', 'gray', 'gray', 'gray', 'brown', 'gray', 'brown', 'gray', 'brown'], ['silver', 'gray', 'gray', 'gray', 'gray', 'silver', 'gray'], ['gray', 'olive'], ['black', 'olive'], ['gray', 'gray'], ['silver'], ['brown', 'brown', 'gray'], ['olive'], ['brown', 'black', 'olive', 'black', 'brown'], ['brown', 'black', 'black', 'black', 'maroon', 'black', 'gray', 'black'], ['gray', 'brown', 'maroon'], [], ['gray', 'gray'], ['brown', 'brown'], ['gray', 'gray'], ['black', 'gray', 'brown', 'gray'], ['silver'], ['gray', 'gray'], ['gray', 'gray'], ['gray', 'black'], ['black', 'gray', 'gray', 'teal', 'black', 'brown', 'black', 'black', 'gray'], ['gray'], ['maroon'], ['black', 'brown', 'gray'], ['olive', 'brown', 'silver'], ['gray'], ['brown', 'gray', 'brown', 'orange'], ['brown', 'black', 'brown', 'black'], ['gray', 'black'], ['gray'], ['brown'], ['gray', 'gray'], ['brown'], ['silver', 'gray'], ['olive'], [], ['silver'], ['black'], ['olive', 'olive', 'brown', 'olive', 'brown'], [], ['brown

In [34]:
# list2 = [['gray olive'], ['black olive']]

# new_list2 = [s[0].split() for s in list2]

# print(new_list2)

[['gray', 'olive'], ['black', 'olive']]


In [89]:
list1 = new_list_l
list2 = new_color_l

# new_list = []

l=list(zip(list1,list2))
print(l)
main_list=[]
for s,c in l:
    lst=[]
    for i in range(len(s)):
        if(s[i].split()[0]=='person'):
            add=s[i]
            pass
        else:
            add=s[i]+ ' ' + c[i]
        lst.append(add)
    str=' '.join(lst)
    main_list.append(str)
main_list

[(['person 60 40 80 100', 'person 20 30 30 80', 'person 10 40 20 80', 'person 80 40 100 100', 'person 50 30 60 100', 'person 40 20 60 90', 'person 30 40 40 90', 'person 70 20 80 50', 'person 30 20 40 80', 'person 60 30 70 100'], ['brown', 'gray', 'gray', 'gray', 'gray', 'brown', 'gray', 'brown', 'gray', 'brown']), (['frisbee 40 50 60 60', 'person 0 0 30 100', 'person 30 20 70 100', 'person 60 0 80 100', 'person 80 40 100 100', 'frisbee 60 20 80 40', 'frisbee 10 50 20 60'], ['silver', 'gray', 'gray', 'gray', 'gray', 'silver', 'gray']), (['car 20 50 40 60', 'train 80 30 80 40'], ['gray', 'olive']), (['stop_sign 30 10 70 60', 'car 30 80 40 90'], ['black', 'olive']), (['person 40 20 100 100', 'person 20 20 60 100'], ['gray', 'gray']), (['bird 20 10 60 60'], ['silver']), (['horse 0 20 60 90', 'person 30 10 40 50', 'person 90 40 100 60'], ['brown', 'brown', 'gray']), (['pizza 0 0 100 100'], ['olive']), (['person 10 10 50 100', 'person 80 20 90 60', 'person 50 20 60 60', 'person 40 20 50 60',

['person 60 40 80 100 person 20 30 30 80 person 10 40 20 80 person 80 40 100 100 person 50 30 60 100 person 40 20 60 90 person 30 40 40 90 person 70 20 80 50 person 30 20 40 80 person 60 30 70 100',
 'frisbee 40 50 60 60 silver person 0 0 30 100 person 30 20 70 100 person 60 0 80 100 person 80 40 100 100 frisbee 60 20 80 40 silver frisbee 10 50 20 60 gray',
 'car 20 50 40 60 gray train 80 30 80 40 olive',
 'stop_sign 30 10 70 60 black car 30 80 40 90 olive',
 'person 40 20 100 100 person 20 20 60 100',
 'bird 20 10 60 60 silver',
 'horse 0 20 60 90 brown person 30 10 40 50 person 90 40 100 60',
 'pizza 0 0 100 100 olive',
 'person 10 10 50 100 person 80 20 90 60 person 50 20 60 60 person 40 20 50 60 cell_phone 30 30 40 50 brown',
 'person 40 60 70 100 person 0 60 30 100 person 90 30 100 100 person 20 50 40 100 person 50 30 70 80 person 80 60 90 100 person 40 40 50 80 person 60 40 80 100',
 'bed 10 10 70 100 gray chair 80 70 90 100 brown dining_table 80 60 100 90 maroon',
 '',
 'person 

In [85]:
str='car 20 50 40 60'
str.split()


['car', '20', '50', '40', '60']

In [90]:
for i in range(len(main_list)):
    main_list[i]=main_list[i].replace("_"," ")
print(main_list)

['person 60 40 80 100 person 20 30 30 80 person 10 40 20 80 person 80 40 100 100 person 50 30 60 100 person 40 20 60 90 person 30 40 40 90 person 70 20 80 50 person 30 20 40 80 person 60 30 70 100', 'frisbee 40 50 60 60 silver person 0 0 30 100 person 30 20 70 100 person 60 0 80 100 person 80 40 100 100 frisbee 60 20 80 40 silver frisbee 10 50 20 60 gray', 'car 20 50 40 60 gray train 80 30 80 40 olive', 'stop sign 30 10 70 60 black car 30 80 40 90 olive', 'person 40 20 100 100 person 20 20 60 100', 'bird 20 10 60 60 silver', 'horse 0 20 60 90 brown person 30 10 40 50 person 90 40 100 60', 'pizza 0 0 100 100 olive', 'person 10 10 50 100 person 80 20 90 60 person 50 20 60 60 person 40 20 50 60 cell phone 30 30 40 50 brown', 'person 40 60 70 100 person 0 60 30 100 person 90 30 100 100 person 20 50 40 100 person 50 30 70 80 person 80 60 90 100 person 40 40 50 80 person 60 40 80 100', 'bed 10 10 70 100 gray chair 80 70 90 100 brown dining table 80 60 100 90 maroon', '', 'person 60 20 100 10

In [91]:
len(main_list)

5000

In [92]:
main_list[2:5]

['car 20 50 40 60 gray train 80 30 80 40 olive',
 'stop sign 30 10 70 60 black car 30 80 40 90 olive',
 'person 40 20 100 100 person 20 20 60 100']

In [66]:
input_list = main_list

main_list_extended = []
for input_str in input_list:
    object_list = input_str.split()
    # print(object_list)
    output_str = ""
    for i, obj in enumerate(object_list):
        # print(i,obj)
        if i % 6 == 0:
            output_str += "object is {} location {} {} {} {} ".format(obj, object_list[i+1], object_list[i+2], object_list[i+3], object_list[i+4])
            # print(output_str)
        elif(i%6==5):
            output_str += "color {} ".format(obj)
            # print('color_str=',output_str)
    main_list_extended.append(output_str)

print(main_list_extended)


['object is person location 60 40 80 100 color brown object is person location 20 30 30 80 color gray object is person location 10 40 20 80 color gray object is person location 80 40 100 100 color gray object is person location 50 30 60 100 color gray object is person location 40 20 60 90 color brown object is person location 30 40 40 90 color gray object is person location 70 20 80 50 color brown object is person location 30 20 40 80 color gray object is person location 60 30 70 100 color brown ', 'object is frisbee location 40 50 60 60 color silver object is person location 0 0 30 100 color gray object is person location 30 20 70 100 color gray object is person location 60 0 80 100 color gray object is person location 80 40 100 100 color gray object is frisbee location 60 20 80 40 color silver object is frisbee location 10 50 20 60 color gray ', 'object is car location 20 50 40 60 color gray object is train location 80 30 80 40 color olive ', 'object is stop_sign location 30 10 70 60

In [69]:
for i in range(len(main_list_extended)):
    main_list_extended[i]=main_list_extended[i].replace("_"," ")
print(main_list_extended[2:5])

['object is car location 20 50 40 60 color gray object is train location 80 30 80 40 color olive ', 'object is stop sign location 30 10 70 60 color black object is car location 30 80 40 90 color olive ', 'object is person location 40 20 100 100 color gray object is person location 20 20 60 100 color gray ']


In [70]:
strings = main_list_extended
with open('../data/x_bert_color_extended.txt', 'w') as file:
    for string in strings:
        file.write(string + '\n')

In [ ]:
# strings = new_list
# with open('../data/x_color.txt', 'w') as file:
#     for string in strings:
#         file.write(string + '\n')

In [93]:
strings = main_list
with open('../data/x_bert_color_not_person.txt', 'w') as file:
    for string in strings:
        file.write(string + '\n')